In [ ]:
%reload_ext autoreload
%autoreload 2

In [2]:
# regular imports
import sys
sys.path.append('..')
import matplotlib.pyplot as plt
%matplotlib inline

# Lightning import 
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import Callback, LearningRateMonitor, ModelCheckpoint
from pytorch_lightning.loggers import WandbLogger
# from pl_bolts.datamodules.mnist_datamodule import MNISTDataModule

# PyTorch imports
import torch
from torch import nn
from torch.nn import functional as F
print(f'Cuda available: {torch.cuda.is_available()}')

import wandb
wandb.login()

# internal imports
from src.callbacks import ImagePredictionLogger
from src.dataset import MNISTDataModule
from src.models import CNN, LitModel
from src.utils import sweep_iteration

Cuda available: True


wandb: Currently logged in as: alkalait (use `wandb login --relogin` to force relogin)


---

In [3]:
# MODEL_CKPT_PATH = '../model/'
# MODEL_CKPT = '../model/model-{epoch:02d}-{val_loss:.2f}'

# checkpoint_callback = ModelCheckpoint(
#     monitor='val_loss',
#     filepath=MODEL_CKPT ,
#     save_top_k=3,
#     mode='min'
# )

In [4]:
proj = 'SimSiam-Lightning'

# Setup datamodule. Comes with its own train / val / test dataloader.
mnist = MNISTDataModule('../data/', batch_size=512)
mnist.prepare_data()
mnist.setup()

cnn = CNN(C=mnist.dims[0], num_classes=mnist.num_classes)  # Architecture
model = LitModel(datamodule=mnist, backbone=cnn, batch_size=512, lr=1e-3, flood=True)
wandb_logger = WandbLogger(project=proj, job_type='train')  # Logger
callbacks = [
    LearningRateMonitor(),  # log the LR
    ImagePredictionLogger(mnist.val_dataloader(batch_size=5000), n_samples=64),
    #early_stop_callback,
]

In [5]:
# cnn

In [6]:
trainer = Trainer(
    max_epochs=200, gpus=-1,  # all GPUs
    logger=wandb_logger, callbacks=callbacks,
    accumulate_grad_batches=1, gradient_clip_val=0,  # 0.5
    progress_bar_refresh_rate=20,
    #checkpoint_callback=checkpoint_callback
    #fast_dev_run=True,
)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


In [7]:
# # Learning rate finder
# lr_finder = trainer.tuner.lr_find(model, num_training=3000, mode='linear', max_lr=1e-2)
# # lr_finder.results  # Results can be found in
# fig = lr_finder.plot(suggest=True)
# lr_finder.suggestion()
# model.hparams.lr = new_lr  # update hparams of the model

In [8]:
trainer.fit(model, mnist)


  | Name     | Type     | Params
--------------------------------------
0 | backbone | CNN      | 158 K 
1 | accuracy | Accuracy | 0     


wandb: WARNING Step must only increase in log calls.  Step 0 < 1; dropping {'lr-Adam': 0.001}.


1

In [9]:
trainer.test()

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': tensor(0.9863, device='cuda:0'),
 'test_loss': tensor(0.0471, device='cuda:0'),
 'train_acc': tensor(0.9941, device='cuda:0'),
 'train_loss': tensor(0.0313, device='cuda:0'),
 'val_acc': tensor(0.9881, device='cuda:0'),
 'val_loss': tensor(0.0515, device='cuda:0')}
--------------------------------------------------------------------------------



[{'train_loss': 0.031338196247816086,
  'train_acc': 0.994140625,
  'val_loss': 0.05150403082370758,
  'val_acc': 0.9880642294883728,
  'test_loss': 0.04709416255354881,
  'test_acc': 0.986328125}]

In [10]:
wandb.finish()

_step,42800
_runtime,1331
_timestamp,1607739352
train_loss,0.03134
train_acc,0.99414
epoch,199
val_loss,0.0515
val_acc,0.98806
lr-Adam,0.0
test_acc,0.98633
test_loss,0.04709


_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_runtime,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_timestamp,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train_loss,█▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_acc,▁▇▇███▇▇▇▇▇█▇██▇█▇▇████▇██████████▇████▇
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
val_loss,█▄▂▃▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_acc,▁▅▇▆▇▆▇▇▇▇▇██▇██████████████████████████
lr-Adam,█████▄▄▄▃▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_acc,▁
test_loss,▁


---

# Hyperparameter sweep 

In [ ]:
# from src.sweeps import sweep_config

# sweep_id = wandb.sweep(sweep_config, project=proj)

In [ ]:
# wandb.agent(sweep_id, function=sweep_iteration, project=proj)

---